In [ ]:
#pragma cling add_include_path("include")
#pragma cling add_library_path("lib")
#pragma cling load("libumpire")

# Umpire Basics

Umpire's `ResourceManager` is the main object used to access all of the objects that Umpire provides. To get a reference to the manager, you can use the `getInstance` method:

In [ ]:
#include "umpire/ResourceManager.hpp"

auto& rm = umpire::ResourceManager::getInstance();

The `ResourceManager` can be used to access `Allocator`s. An `Allocator` is an object that is used to allocate, deallocate, and introspect memory.

In [ ]:
auto alloc = rm.getAllocator("HOST");

In the previous cell, we grabbed the "HOST" `Allocator`. Depneding on how your system is configured, Umpire will create a number of predefined allocators for the available memory resources. The list of predefined Allocators is:

- `"HOST"`: available on all systems, provides access to regular DDR memory.
- `"DEVICE"`: available with CUDA or HIP, provides access to GPU memory.
- `"UM"`: available with CUDA, provides "unified" memory accesibly on the host or GPU.
- `"PINNED"`: avaible with CUDA or HIP, provides access to pinned DDR memory.

_N.B.: since this tutorial is running with a CPU-only build, only the HOST resource is available._

## Allocating Memory

You can allocate and deallocate memory using the `allocate` and `deallocate` methods. Allocation sizes are in bytes, and return void pointers, which you can cast to the appropriate type:

In [ ]:
double* data = static_cast<double*>(alloc.allocate(1024*sizeof(double)));

std::cout << "data allocated at: " << data << std::endl;

alloc.deallocate(data);

### Typed Allocators

`TypedAllocators` are available in Umpire's C++ API, and allow you to create an `Allocato`r for a specific data type:

In [ ]:
#include "umpire/TypedAllocator.hpp"

auto double_allocator = umpire::TypedAllocator<double>{alloc};

data = double_allocator.allocate(10); // allocate 10 doubles

std::cout << "size of data is: " << alloc.getSize(data) << std::endl;

double_allocator.deallocate(data, 10);

# Allocation Strategies

Allocation strategies decouple how & where data is allocated, allowing you to build up complex mechanisms for allocating data. 


## Pool

Let's start with a simple example and create a DynamicPool:

In [ ]:
#include "umpire/strategy/DynamicPool.hpp"

auto pool = rm.makeAllocator<umpire::strategy::DynamicPool, false>("pool", alloc);

data = static_cast<double*>(pool.allocate(1024));

std::cout << "data allocated at: " << data << std::endl;

pool.deallocate(data);

data = static_cast<double*>(pool.allocate(4096));

std::cout << "data allocated (again) at: " << data << std::endl;

pool.deallocate(data);

data = static_cast<double*>(pool.allocate(2048));

You can query the current size of the pool (total size of all allocations), as well as the _actual_ size (how much memory the pool has allocated):

In [ ]:
std::cout << pool.getCurrentSize() << std::endl;

In [ ]:
std::cout << pool.getActualSize() << std::endl;

The pool will hold on to memory even when there are no active allocations. This is what makes it fast:

In [ ]:
pool.deallocate(data);
std::cout << pool.getActualSize() << std::endl;
std::cout << pool.getCurrentSize() << std::endl;

To free all the memory the pool is holding on to, you can use the `release` method:

In [ ]:
pool.release();
std::cout << pool.getActualSize() << std::endl;
std::cout << pool.getCurrentSize() << std::endl;

## SizeLimiter

Let's look at some other strategies, and how these can be combined. Umpire provides a `SizeLimiter` strategy that limits the total size of allocations. We can combine this with a pool to create a pooled allocator than can never be above a certain size:

In [ ]:
#include "umpire/strategy/SizeLimiter.hpp"

auto size_limiter =  rm.makeAllocator<umpire::strategy::SizeLimiter>("size_limiter", alloc, 1024*1024);
auto limited_pool = rm.makeAllocator<umpire::strategy::DynamicPool>("limited_pool", size_limiter, 1024*1024, 1024);

The pool will be created with 1Mb pre-allocated. Trying to allocate more than this will cause the allocations to fail:

In [ ]:
data = static_cast<double*>(limited_pool.allocate(1024));

try {
    void* will_fail = limited_pool.allocate(1024*1024);
} catch (...) {
    std::cout << "Uh oh" << std::endl;
}

limited_pool.deallocate(data);
limited_pool.release();

# Sandbox

In the cell below, try editing and playing around with Umpire. If you are in a tutorial session with us, please ask if you have any questions. If not, email umpire-dev@llnl.gov


In [ ]:
// rm is the ResourceManager
// alloc is the "HOST" allocator
// pool is a DynamicPool
// size_limiter is an allocator limited to 1MB total
// limited_pool is a pool built on this allocator
{
    auto my_data = alloc.allocate();

}